In [1]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import _tree
from sklearn.preprocessing import LabelEncoder
import gradio as gr

le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}

engine = pyttsx3.init()

# Function to calculate the overall severity of the symptom
def calc_condition(exp, days):
    total_severity = sum(severityDictionary.get(item, 0) for item in exp)
    average_severity = (total_severity * days) / (len(exp) + 1)
    if average_severity > 13:
        return "You should take a consultation with a doctor."
    else:
        return "It might not be that bad but you should take precautions."

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    with open("/content/drive/MyDrive/ana/tp/symptom_Description.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            description_list[row[0]] = row[1]

    with open("/content/drive/MyDrive/ana/tp/Symptom_severity.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            if len(row) < 2: continue
            try:
                severityDictionary[row[0]] = int(row[1])
            except ValueError:
                print(f"Warning: Could not convert severity for {row[0]} to integer.")
    
    with open("/content/drive/MyDrive/ana/tp/symptom_precaution.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            precautionDictionary[row[0]] = row[1:]

# Secondary prediction function
def sec_predict(symptoms_exp):
    df = pd.read_csv('/content/drive/MyDrive/ana/tp/Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[symptoms_dict[item]] = 1

    return rf_clf.predict([input_vector])

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    df = pd.read_csv('/content/drive/MyDrive/ana/tp/Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    
    symptoms_exp = symptom_input.split(", ")
    second_prediction = sec_predict(symptoms_exp)
    severity_message = calc_condition(symptoms_exp, int(num_days))
    
    predicted_disease = le.inverse_transform([clf.predict([np.isin(X.columns, symptoms_exp).astype(int)])[0]])[0]
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    response = f"""
    Hello {name},
    Based on your symptoms, you may have: *{predicted_disease}*
    
    *Description*: {description}
    
    *Precautions*:
    {"; ".join(precautions)}
    
    *Severity Advice*: {severity_message}
    """
    return response

# Load dictionaries and fit label encoder
load_data()
le.fit(df['prognosis'])

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.inputs.Textbox(label="Your Name"),
        gr.inputs.Textbox(label="List of Symptoms (separate by comma)"),
        gr.inputs.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)

interface.launch()

ModuleNotFoundError: No module named 'gradio'

In [3]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement gradio (from versions: none)
ERROR: No matching distribution found for gradio


In [1]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr
from fuzzywuzzy import process  # For fuzzy matching

# Initialize label encoder and dictionaries
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}
engine = pyttsx3.init()

# Function to calculate the overall severity and get the disease description and precautions
def calc_condition(symptoms_exp, days, predicted_disease):
    total_severity = 0
    for symptom in symptoms_exp:
        total_severity += severityDictionary.get(symptom, 0)
    
    average_severity = (total_severity * days) / (len(symptoms_exp) + 1)
    
    # Fetch the description and precautions for the predicted disease
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    if average_severity > 13:
        severity_message = "You should take a consultation with a doctor."
    else:
        severity_message = "It might not be that bad, but you should take precautions."
    
    return total_severity, average_severity, description, precautions, severity_message

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    try:
        with open("/content/drive/MyDrive/ana/tp/symptom_Description.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                description_list[row[0]] = row[1]

        with open("/content/drive/MyDrive/ana/tp/Symptom_severity.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                if len(row) < 2: continue
                try:
                    severityDictionary[row[0]] = int(row[1])
                except ValueError:
                    print(f"Warning: Could not convert severity for {row[0]} to integer.")

        with open("/content/drive/MyDrive/ana/tp/symptom_precaution.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                precautionDictionary[row[0]] = row[1:]
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV data: {e}")

# Load the data
load_data()

# Load training data
df = pd.read_csv('/content/drive/MyDrive/ana/tp/Training.csv')
le.fit(df['prognosis'])

# Secondary prediction function (for secondary symptoms)
def sec_predict(symptoms_exp):
    X = df.iloc[:, :-1]  # All columns except 'prognosis'
    y = df['prognosis']  # Target column (disease prognosis)
    
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    symptoms_exp_normalized = [symptom.strip().lower() for symptom in symptoms_exp]
    
    for item in symptoms_exp_normalized:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            matches = process.extract(item, symptoms_dict.keys(), limit=3)
            for match, score in matches:
                if score >= 70:
                    input_vector[symptoms_dict[match]] = 1
                    break
    
    prediction = clf.predict([input_vector])
    return prediction[0]

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    try:
        symptoms_exp = [symptom.strip().lower() for symptom in symptom_input.split(", ")]
        
        X = df.iloc[:, :-1]
        symptoms_columns = [symptom.strip().lower() for symptom in X.columns]
        
        unmatched_symptoms = [symptom for symptom in symptoms_exp if symptom not in symptoms_columns]
        
        partial_matches = {}
        if unmatched_symptoms:
            for symptom in unmatched_symptoms:
                matches = process.extract(symptom, symptoms_columns, limit=3)
                partial_matches[symptom] = matches
        
        warning_message = f"Warning: Some symptoms did not match exactly. Here are some partial matches:\n" if partial_matches else "All symptoms match exactly."
        for symptom, matches in partial_matches.items():
            match_details = [f"{match} ({score}%)" for match, score in matches]
            warning_message += f" - {symptom.capitalize()}: {', '.join(match_details)}\n"
        
        second_prediction = sec_predict(symptoms_exp)
        
        X_train, X_test, y_train, y_test = train_test_split(X, df['prognosis'], test_size=0.3, random_state=20, stratify=df['prognosis'])
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
        symptoms_vector = np.isin(X.columns.str.lower(), symptoms_exp).astype(int)
        
        predicted_index = clf.predict([symptoms_vector])[0]
        
        try:
            predicted_disease = le.inverse_transform([predicted_index])[0]
        except ValueError:
            predicted_disease = "Unknown disease"
        
        # Fetch description, precautions, and severity
        total_severity, average_severity, description, precautions, severity_message = calc_condition(symptoms_exp, int(num_days), predicted_disease)
        
        # Format the response
        response = f"""
        Hello {name},
        Based on your symptoms, you may have: {predicted_disease}

        Description: {description}

        Total Severity Score: {total_severity}
        Average Severity over {num_days} days: {average_severity}

        Secondary Prediction (based on fuzzy matching): {second_prediction}

        Precautions: {"; ".join(precautions)}

        Severity Advice: {severity_message}
        """
        
        response += warning_message
        
        return response
    except Exception as e:
        return f"Error: Unable to predict disease. Please check the symptoms. ({str(e)})"

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Textbox(label="List of Symptoms (separate by comma)"),
        gr.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)

interface.launch()

ModuleNotFoundError: No module named 'gradio'

In [7]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [4]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr
from fuzzywuzzy import process  # For fuzzy matching

# Initialize label encoder and dictionaries
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}
engine = pyttsx3.init()

# Function to calculate the overall severity and get the disease description and precautions
def calc_condition(symptoms_exp, days, predicted_disease):
    total_severity = 0
    for symptom in symptoms_exp:
        total_severity += severityDictionary.get(symptom, 0)
    
    average_severity = (total_severity * days) / (len(symptoms_exp) + 1)
    
    # Fetch the description and precautions for the predicted disease
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    if average_severity > 13:
        severity_message = "You should take a consultation with a doctor."
    else:
        severity_message = "It might not be that bad, but you should take precautions."
    
    return total_severity, average_severity, description, precautions, severity_message

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    try:
        with open("/content/drive/MyDrive/ana/tp/symptom_Description.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                description_list[row[0]] = row[1]

        with open("/content/drive/MyDrive/ana/tp/Symptom_severity.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                if len(row) < 2: continue
                try:
                    severityDictionary[row[0]] = int(row[1])
                except ValueError:
                    print(f"Warning: Could not convert severity for {row[0]} to integer.")

        with open("/content/drive/MyDrive/ana/tp/symptom_precaution.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                precautionDictionary[row[0]] = row[1:]
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV data: {e}")

# Load the data
load_data()

# Load training data
df = pd.read_csv('/content/drive/MyDrive/ana/tp/Training.csv')
le.fit(df['prognosis'])

# Secondary prediction function (for secondary symptoms)
def sec_predict(symptoms_exp):
    X = df.iloc[:, :-1]  # All columns except 'prognosis'
    y = df['prognosis']  # Target column (disease prognosis)
    
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    symptoms_exp_normalized = [symptom.strip().lower() for symptom in symptoms_exp]
    
    for item in symptoms_exp_normalized:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            matches = process.extract(item, symptoms_dict.keys(), limit=3)
            for match, score in matches:
                if score >= 70:
                    input_vector[symptoms_dict[match]] = 1
                    break
    
    prediction = clf.predict([input_vector])
    return prediction[0]

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    try:
        symptoms_exp = [symptom.strip().lower() for symptom in symptom_input.split(", ")]
        
        X = df.iloc[:, :-1]
        symptoms_columns = [symptom.strip().lower() for symptom in X.columns]
        
        unmatched_symptoms = [symptom for symptom in symptoms_exp if symptom not in symptoms_columns]
        
        partial_matches = {}
        if unmatched_symptoms:
            for symptom in unmatched_symptoms:
                matches = process.extract(symptom, symptoms_columns, limit=3)
                partial_matches[symptom] = matches
        
        warning_message = f"Warning: Some symptoms did not match exactly. Here are some partial matches:\n" if partial_matches else "All symptoms match exactly."
        for symptom, matches in partial_matches.items():
            match_details = [f"{match} ({score}%)" for match, score in matches]
            warning_message += f" - {symptom.capitalize()}: {', '.join(match_details)}\n"
        
        second_prediction = sec_predict(symptoms_exp)
        
        X_train, X_test, y_train, y_test = train_test_split(X, df['prognosis'], test_size=0.3, random_state=20, stratify=df['prognosis'])
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
        symptoms_vector = np.isin(X.columns.str.lower(), symptoms_exp).astype(int)
        
        predicted_index = clf.predict([symptoms_vector])[0]
        
        try:
            predicted_disease = le.inverse_transform([predicted_index])[0]
        except ValueError:
            predicted_disease = "Unknown disease"
        
        # Fetch description, precautions, and severity
        total_severity, average_severity, description, precautions, severity_message = calc_condition(symptoms_exp, int(num_days), predicted_disease)
        
        # Format the response
        response = f"""
        Hello {name},
        Based on your symptoms, you may have: {predicted_disease}

        Description: {description}

        Total Severity Score: {total_severity}
        Average Severity over {num_days} days: {average_severity}

        Secondary Prediction (based on fuzzy matching): {second_prediction}

        Precautions: {"; ".join(precautions)}

        Severity Advice: {severity_message}
        """
        
        response += warning_message
        
        return response
    except Exception as e:
        return f"Error: Unable to predict disease. Please check the symptoms. ({str(e)})"

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Textbox(label="List of Symptoms (separate by comma)"),
        gr.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)

interface.launch()

ModuleNotFoundError: No module named 'gradio'

In [8]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr
from fuzzywuzzy import process  # For fuzzy matching

# Initialize label encoder and dictionaries
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}
engine = pyttsx3.init()

# Function to calculate the overall severity and get the disease description and precautions
def calc_condition(symptoms_exp, days, predicted_disease):
    total_severity = 0
    for symptom in symptoms_exp:
        total_severity += severityDictionary.get(symptom, 0)
    
    average_severity = (total_severity * days) / (len(symptoms_exp) + 1)
    
    # Fetch the description and precautions for the predicted disease
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    if average_severity > 13:
        severity_message = "You should take a consultation with a doctor."
    else:
        severity_message = "It might not be that bad, but you should take precautions."
    
    return total_severity, average_severity, description, precautions, severity_message

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    try:
        with open("/content/drive/MyDrive/ana/tp/symptom_Description.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                description_list[row[0]] = row[1]

        with open("/content/drive/MyDrive/ana/tp/Symptom_severity.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                if len(row) < 2: continue
                try:
                    severityDictionary[row[0]] = int(row[1])
                except ValueError:
                    print(f"Warning: Could not convert severity for {row[0]} to integer.")

        with open("/content/drive/MyDrive/ana/tp/symptom_precaution.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                precautionDictionary[row[0]] = row[1:]
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV data: {e}")

# Load the data
load_data()

# Load training data
df = pd.read_csv('/content/drive/MyDrive/ana/tp/Training.csv')
le.fit(df['prognosis'])

# Secondary prediction function (for secondary symptoms)
def sec_predict(symptoms_exp):
    X = df.iloc[:, :-1]  # All columns except 'prognosis'
    y = df['prognosis']  # Target column (disease prognosis)
    
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    symptoms_exp_normalized = [symptom.strip().lower() for symptom in symptoms_exp]
    
    for item in symptoms_exp_normalized:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            matches = process.extract(item, symptoms_dict.keys(), limit=3)
            for match, score in matches:
                if score >= 70:
                    input_vector[symptoms_dict[match]] = 1
                    break
    
    prediction = clf.predict([input_vector])
    return prediction[0]

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    try:
        symptoms_exp = [symptom.strip().lower() for symptom in symptom_input.split(", ")]
        
        X = df.iloc[:, :-1]
        symptoms_columns = [symptom.strip().lower() for symptom in X.columns]
        
        unmatched_symptoms = [symptom for symptom in symptoms_exp if symptom not in symptoms_columns]
        
        partial_matches = {}
        if unmatched_symptoms:
            for symptom in unmatched_symptoms:
                matches = process.extract(symptom, symptoms_columns, limit=3)
                partial_matches[symptom] = matches
        
        warning_message = f"Warning: Some symptoms did not match exactly. Here are some partial matches:\n" if partial_matches else "All symptoms match exactly."
        for symptom, matches in partial_matches.items():
            match_details = [f"{match} ({score}%)" for match, score in matches]
            warning_message += f" - {symptom.capitalize()}: {', '.join(match_details)}\n"
        
        second_prediction = sec_predict(symptoms_exp)
        
        X_train, X_test, y_train, y_test = train_test_split(X, df['prognosis'], test_size=0.3, random_state=20, stratify=df['prognosis'])
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
        symptoms_vector = np.isin(X.columns.str.lower(), symptoms_exp).astype(int)
        
        predicted_index = clf.predict([symptoms_vector])[0]
        
        try:
            predicted_disease = le.inverse_transform([predicted_index])[0]
        except ValueError:
            predicted_disease = "Unknown disease"
        
        # Fetch description, precautions, and severity
        total_severity, average_severity, description, precautions, severity_message = calc_condition(symptoms_exp, int(num_days), predicted_disease)
        
        # Format the response
        response = f"""
        Hello {name},
        Based on your symptoms, you may have: {predicted_disease}

        Description: {description}

        Total Severity Score: {total_severity}
        Average Severity over {num_days} days: {average_severity}

        Secondary Prediction (based on fuzzy matching): {second_prediction}

        Precautions: {"; ".join(precautions)}

        Severity Advice: {severity_message}
        """
        
        response += warning_message
        
        return response
    except Exception as e:
        return f"Error: Unable to predict disease. Please check the symptoms. ({str(e)})"

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Textbox(label="List of Symptoms (separate by comma)"),
        gr.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)

interface.launch()

ModuleNotFoundError: No module named 'fuzzywuzzy'

In [3]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr
from fuzzywuzzy import process  # For fuzzy matching

# Initialize label encoder and dictionaries
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}
engine = pyttsx3.init()

# Function to calculate the overall severity and get the disease description and precautions
def calc_condition(symptoms_exp, days, predicted_disease):
    total_severity = 0
    for symptom in symptoms_exp:
        total_severity += severityDictionary.get(symptom, 0)
    
    average_severity = (total_severity * days) / (len(symptoms_exp) + 1)
    
    # Fetch the description and precautions for the predicted disease
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    if average_severity > 13:
        severity_message = "You should take a consultation with a doctor."
    else:
        severity_message = "It might not be that bad, but you should take precautions."
    
    return total_severity, average_severity, description, precautions, severity_message

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    try:
        with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                description_list[row[0]] = row[1]

        with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                if len(row) < 2: continue
                try:
                    severityDictionary[row[0]] = int(row[1])
                except ValueError:
                    print(f"Warning: Could not convert severity for {row[0]} to integer.")

        with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                precautionDictionary[row[0]] = row[1:]
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV data: {e}")

# Load the data
load_data()

# Load training data
df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
le.fit(df['prognosis'])

# Secondary prediction function (for secondary symptoms)
def sec_predict(symptoms_exp):
    X = df.iloc[:, :-1]  # All columns except 'prognosis'
    y = df['prognosis']  # Target column (disease prognosis)
    
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    symptoms_exp_normalized = [symptom.strip().lower() for symptom in symptoms_exp]
    
    for item in symptoms_exp_normalized:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            matches = process.extract(item, symptoms_dict.keys(), limit=3)
            for match, score in matches:
                if score >= 70:
                    input_vector[symptoms_dict[match]] = 1
                    break
    
    prediction = clf.predict([input_vector])
    return prediction[0]

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    try:
        symptoms_exp = [symptom.strip().lower() for symptom in symptom_input.split(", ")]
        
        X = df.iloc[:, :-1]
        symptoms_columns = [symptom.strip().lower() for symptom in X.columns]
        
        unmatched_symptoms = [symptom for symptom in symptoms_exp if symptom not in symptoms_columns]
        
        partial_matches = {}
        if unmatched_symptoms:
            for symptom in unmatched_symptoms:
                matches = process.extract(symptom, symptoms_columns, limit=3)
                partial_matches[symptom] = matches
        
        warning_message = f"Warning: Some symptoms did not match exactly. Here are some partial matches:\n" if partial_matches else "All symptoms match exactly."
        for symptom, matches in partial_matches.items():
            match_details = [f"{match} ({score}%)" for match, score in matches]
            warning_message += f" - {symptom.capitalize()}: {', '.join(match_details)}\n"
        
        second_prediction = sec_predict(symptoms_exp)
        
        X_train, X_test, y_train, y_test = train_test_split(X, df['prognosis'], test_size=0.3, random_state=20, stratify=df['prognosis'])
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
        symptoms_vector = np.isin(X.columns.str.lower(), symptoms_exp).astype(int)
        
        predicted_index = clf.predict([symptoms_vector])[0]
        
        try:
            predicted_disease = le.inverse_transform([predicted_index])[0]
        except ValueError:
            predicted_disease = "Unknown disease"
        
        # Fetch description, precautions, and severity
        total_severity, average_severity, description, precautions, severity_message = calc_condition(symptoms_exp, int(num_days), predicted_disease)
        
        # Format the response
        response = f"""
        Hello {name},
        Based on your symptoms, you may have: {predicted_disease}

        Description: {description}

        Total Severity Score: {total_severity}
        Average Severity over {num_days} days: {average_severity}

        Secondary Prediction (based on fuzzy matching): {second_prediction}

        Precautions: {"; ".join(precautions)}

        Severity Advice: {severity_message}
        """
        
        response += warning_message
        
        return response
    except Exception as e:
        return f"Error: Unable to predict disease. Please check the symptoms. ({str(e)})"

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Textbox(label="List of Symptoms (separate by comma)"),
        gr.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)

interface.launch()

Data loaded successfully.
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [9]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr
from fuzzywuzzy import process  # For fuzzy matching

# Initialize label encoder and dictionaries
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}
engine = pyttsx3.init()

# Function to calculate the overall severity and get the disease description and precautions
def calc_condition(symptoms_exp, days, predicted_disease):
    total_severity = 0
    for symptom in symptoms_exp:
        total_severity += severityDictionary.get(symptom, 0)
    
    average_severity = (total_severity * days) / (len(symptoms_exp) + 1)
    
    # Fetch the description and precautions for the predicted disease
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    if average_severity > 13:
        severity_message = "You should take a consultation with a doctor."
    else:
        severity_message = "It might not be that bad, but you should take precautions."
    
    return total_severity, average_severity, description, precautions, severity_message

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    try:
        with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                description_list[row[0]] = row[1]

        with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                if len(row) < 2: continue
                try:
                    severityDictionary[row[0]] = int(row[1])
                except ValueError:
                    print(f"Warning: Could not convert severity for {row[0]} to integer.")

        with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                precautionDictionary[row[0]] = row[1:]
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV data: {e}")

# Load the data
load_data()

# Load training data
df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
le.fit(df['prognosis'])

# Secondary prediction function (for secondary symptoms)
def sec_predict(symptoms_exp):
    X = df.iloc[:, :-1]  # All columns except 'prognosis'
    y = df['prognosis']  # Target column (disease prognosis)
    
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    symptoms_exp_normalized = [symptom.strip().lower() for symptom in symptoms_exp]
    
    for item in symptoms_exp_normalized:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            matches = process.extract(item, symptoms_dict.keys(), limit=3)
            for match, score in matches:
                if score >= 70:
                    input_vector[symptoms_dict[match]] = 1
                    break
    
    prediction = clf.predict([input_vector])
    return prediction[0]

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    try:
        symptoms_exp = [symptom.strip().lower() for symptom in symptom_input.split(", ")]
        
        X = df.iloc[:, :-1]
        symptoms_columns = [symptom.strip().lower() for symptom in X.columns]
        
        unmatched_symptoms = [symptom for symptom in symptoms_exp if symptom not in symptoms_columns]
        
        partial_matches = {}
        if unmatched_symptoms:
            for symptom in unmatched_symptoms:
                matches = process.extract(symptom, symptoms_columns, limit=3)
                partial_matches[symptom] = matches
        
        warning_message = f"Warning: Some symptoms did not match exactly. Here are some partial matches:\n" if partial_matches else "All symptoms match exactly."
        for symptom, matches in partial_matches.items():
            match_details = [f"{match} ({score}%)" for match, score in matches]
            warning_message += f" - {symptom.capitalize()}: {', '.join(match_details)}\n"
        
        second_prediction = sec_predict(symptoms_exp)
        
        X_train, X_test, y_train, y_test = train_test_split(X, df['prognosis'], test_size=0.3, random_state=20, stratify=df['prognosis'])
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
        symptoms_vector = np.isin(X.columns.str.lower(), symptoms_exp).astype(int)
        
        predicted_index = clf.predict([symptoms_vector])[0]
        
        try:
            predicted_disease = le.inverse_transform([predicted_index])[0]
        except ValueError:
            predicted_disease = "Unknown disease"
        
        # Fetch description, precautions, and severity
        total_severity, average_severity, description, precautions, severity_message = calc_condition(symptoms_exp, int(num_days), predicted_disease)
        
        # Format the response
        response = f"""
        Hello {name},
        Based on your symptoms, you may have: {predicted_disease}

        Description: {description}

        Total Severity Score: {total_severity}
        Average Severity over {num_days} days: {average_severity}

        Secondary Prediction (based on fuzzy matching): {second_prediction}

        Precautions: {"; ".join(precautions)}

        Severity Advice: {severity_message}
        """
        
        response += warning_message
        
        return response
    except Exception as e:
        return f"Error: Unable to predict disease. Please check the symptoms. ({str(e)})"

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Textbox(label="List of Symptoms (separate by comma)"),
        gr.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)

interface.launch()

Data loaded successfully.
* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [11]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr
from fuzzywuzzy import process  # For fuzzy matching

# Initialize label encoder and dictionaries
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}
engine = pyttsx3.init()

# Function to calculate the overall severity and get the disease description and precautions
def calc_condition(symptoms_exp, days, predicted_disease):
    total_severity = 0
    for symptom in symptoms_exp:
        total_severity += severityDictionary.get(symptom, 0)
    
    average_severity = (total_severity * days) / (len(symptoms_exp) + 1)
    
    # Fetch the description and precautions for the predicted disease
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    if average_severity > 13:
        severity_message = "You should take a consultation with a doctor."
    else:
        severity_message = "It might not be that bad, but you should take precautions."
    
    return total_severity, average_severity, description, precautions, severity_message

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    try:
        with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                description_list[row[0]] = row[1]

        with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                if len(row) < 2: continue
                try:
                    severityDictionary[row[0]] = int(row[1])
                except ValueError:
                    print(f"Warning: Could not convert severity for {row[0]} to integer.")

        with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                precautionDictionary[row[0]] = row[1:]
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV data: {e}")

# Load the data
load_data()

# Load training data
df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
le.fit(df['prognosis'])

# Secondary prediction function (for secondary symptoms)
def sec_predict(symptoms_exp):
    X = df.iloc[:, :-1]  # All columns except 'prognosis'
    y = df['prognosis']  # Target column (disease prognosis)
    
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    symptoms_exp_normalized = [symptom.strip().lower() for symptom in symptoms_exp]
    
    for item in symptoms_exp_normalized:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            matches = process.extract(item, symptoms_dict.keys(), limit=3)
            for match, score in matches:
                if score >= 70:
                    input_vector[symptoms_dict[match]] = 1
                    break
    
    prediction = clf.predict([input_vector])
    return prediction[0]

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    try:
        symptoms_exp = [symptom.strip().lower() for symptom in symptom_input.split(", ")]
        
        X = df.iloc[:, :-1]
        symptoms_columns = [symptom.strip().lower() for symptom in X.columns]
        
        unmatched_symptoms = [symptom for symptom in symptoms_exp if symptom not in symptoms_columns]
        
        partial_matches = {}
        if unmatched_symptoms:
            for symptom in unmatched_symptoms:
                matches = process.extract(symptom, symptoms_columns, limit=3)
                partial_matches[symptom] = matches
        
        warning_message = f"Warning: Some symptoms did not match exactly. Here are some partial matches:\n" if partial_matches else "All symptoms match exactly."
        for symptom, matches in partial_matches.items():
            match_details = [f"{match} ({score}%)" for match, score in matches]
            warning_message += f" - {symptom.capitalize()}: {', '.join(match_details)}\n"
        
        second_prediction = sec_predict(symptoms_exp)
        
        X_train, X_test, y_train, y_test = train_test_split(X, df['prognosis'], test_size=0.3, random_state=20, stratify=df['prognosis'])
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
        symptoms_vector = np.isin(X.columns.str.lower(), symptoms_exp).astype(int)
        
        predicted_index = clf.predict([symptoms_vector])[0]
        
        try:
            predicted_disease = le.inverse_transform([predicted_index])[0]
        except ValueError:
            predicted_disease = "Unknown disease"
        
        # Fetch description, precautions, and severity
        total_severity, average_severity, description, precautions, severity_message = calc_condition(symptoms_exp, int(num_days), predicted_disease)
        
        # Format the response
        response = f"""
        Hello {name},
        Based on your symptoms, you may have: {predicted_disease}

        Description: {description}

        Total Severity Score: {total_severity}
        Average Severity over {num_days} days: {average_severity}

        Secondary Prediction (based on fuzzy matching): {second_prediction}

        Precautions: {"; ".join(precautions)}

        Severity Advice: {severity_message}
        """
        
        response += warning_message
        
        return response
    except Exception as e:
        return f"Error: Unable to predict disease. Please check the symptoms. ({str(e)})"

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Textbox(label="List of Symptoms (separate by comma)"),
        gr.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)

interface.launch()


Data loaded successfully.
* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [13]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr
from fuzzywuzzy import process  # For fuzzy matching

# Initialize label encoder and dictionaries
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}
engine = pyttsx3.init()

# Function to calculate the overall severity and get the disease description and precautions
def calc_condition(symptoms_exp, days, predicted_disease):
    total_severity = 0
    for symptom in symptoms_exp:
        total_severity += severityDictionary.get(symptom, 0)
    
    average_severity = (total_severity * days) / (len(symptoms_exp) + 1)
    
    # Fetch the description and precautions for the predicted disease
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    if average_severity > 13:
        severity_message = "You should take a consultation with a doctor."
    else:
        severity_message = "It might not be that bad, but you should take precautions."
    
    return total_severity, average_severity, description, precautions, severity_message

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    try:
        with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                description_list[row[0]] = row[1]

        with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                if len(row) < 2: continue
                try:
                    severityDictionary[row[0]] = int(row[1])
                except ValueError:
                    print(f"Warning: Could not convert severity for {row[0]} to integer.")

        with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                precautionDictionary[row[0]] = row[1:]
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV data: {e}")

# Load the data
load_data()

# Load training data
df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
le.fit(df['prognosis'])

# Secondary prediction function (for secondary symptoms)
def sec_predict(symptoms_exp):
    X = df.iloc[:, :-1]  # All columns except 'prognosis'
    y = df['prognosis']  # Target column (disease prognosis)
    
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    symptoms_exp_normalized = [symptom.strip().lower() for symptom in symptoms_exp]
    
    for item in symptoms_exp_normalized:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            matches = process.extract(item, symptoms_dict.keys(), limit=3)
            for match, score in matches:
                if score >= 70:
                    input_vector[symptoms_dict[match]] = 1
                    break
    
    prediction = clf.predict([input_vector])
    return prediction[0]

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    try:
        symptoms_exp = [symptom.strip().lower() for symptom in symptom_input.split(", ")]
        
        X = df.iloc[:, :-1]
        symptoms_columns = [symptom.strip().lower() for symptom in X.columns]
        
        unmatched_symptoms = [symptom for symptom in symptoms_exp if symptom not in symptoms_columns]
        
        partial_matches = {}
        if unmatched_symptoms:
            for symptom in unmatched_symptoms:
                matches = process.extract(symptom, symptoms_columns, limit=3)
                partial_matches[symptom] = matches
        
        warning_message = f"Warning: Some symptoms did not match exactly. Here are some partial matches:\n" if partial_matches else "All symptoms match exactly."
        for symptom, matches in partial_matches.items():
            match_details = [f"{match} ({score}%)" for match, score in matches]
            warning_message += f" - {symptom.capitalize()}: {', '.join(match_details)}\n"
        
        second_prediction = sec_predict(symptoms_exp)
        
        X_train, X_test, y_train, y_test = train_test_split(X, df['prognosis'], test_size=0.3, random_state=20, stratify=df['prognosis'])
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
        symptoms_vector = np.isin(X.columns.str.lower(), symptoms_exp).astype(int)
        
        predicted_index = clf.predict([symptoms_vector])[0]
        
        try:
            predicted_disease = le.inverse_transform([predicted_index])[0]
        except ValueError:
            predicted_disease = "Unknown disease"
        
        # Fetch description, precautions, and severity
        total_severity, average_severity, description, precautions, severity_message = calc_condition(symptoms_exp, int(num_days), predicted_disease)
        
        # Format the response
        response = f"""
        Hello {name},
        Based on your symptoms, you may have: {predicted_disease}

        Description: {description}

        Total Severity Score: {total_severity}
        Average Severity over {num_days} days: {average_severity}

        Secondary Prediction (based on fuzzy matching): {second_prediction}

        Precautions: {"; ".join(precautions)}

        Severity Advice: {severity_message}
        """
        
        response += warning_message
        
        return response
    except Exception as e:
        return f"Error: Unable to predict disease. Please check the symptoms. ({str(e)})"

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Textbox(label="List of Symptoms (separate by comma)"),
        gr.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)

interface.launch()


Data loaded successfully.
* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [7]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr
from fuzzywuzzy import process  # For fuzzy matching

# Initialize label encoder and dictionaries
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}
engine = pyttsx3.init()

# Function to calculate the overall severity and get the disease description and precautions
def calc_condition(symptoms_exp, days, predicted_disease):
    total_severity = 0
    for symptom in symptoms_exp:
        total_severity += severityDictionary.get(symptom, 0)
    
    average_severity = (total_severity * days) / (len(symptoms_exp) + 1)
    
    # Fetch the description and precautions for the predicted disease
    description = description_list.get(predicted_disease, "Description not available.")
    precautions = precautionDictionary.get(predicted_disease, ["No specific precautions found."])
    
    if average_severity > 13:
        severity_message = "You should take a consultation with a doctor."
    else:
        severity_message = "It might not be that bad, but you should take precautions."
    
    return total_severity, average_severity, description, precautions, severity_message

# Load data from CSV files
def load_data():
    global severityDictionary, description_list, precautionDictionary
    try:
        with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                description_list[row[0]] = row[1]

        with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                if len(row) < 2: continue
                try:
                    severityDictionary[row[0]] = int(row[1])
                except ValueError:
                    print(f"Warning: Could not convert severity for {row[0]} to integer.")

        with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                precautionDictionary[row[0]] = row[1:]
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV data: {e}")

# Load the data
load_data()

# Load training data
df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
le.fit(df['prognosis'])

# Secondary prediction function (for secondary symptoms)
def sec_predict(symptoms_exp):
    X = df.iloc[:, :-1]  # All columns except 'prognosis'
    y = df['prognosis']  # Target column (disease prognosis)
    
    clf = DecisionTreeClassifier()
    clf.fit(X, y)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    symptoms_exp_normalized = [symptom.strip().lower() for symptom in symptoms_exp]
    
    for item in symptoms_exp_normalized:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            matches = process.extract(item, symptoms_dict.keys(), limit=3)
            for match, score in matches:
                if score >= 70:
                    input_vector[symptoms_dict[match]] = 1
                    break
    
    prediction = clf.predict([input_vector])
    return prediction[0]

# Main chatbot function
def healthcare_chatbot(name, symptom_input, num_days):
    try:
        symptoms_exp = [symptom.strip().lower() for symptom in symptom_input.split(", ")]
        
        X = df.iloc[:, :-1]
        symptoms_columns = [symptom.strip().lower() for symptom in X.columns]
        
        unmatched_symptoms = [symptom for symptom in symptoms_exp if symptom not in symptoms_columns]
        
        partial_matches = {}
        if unmatched_symptoms:
            for symptom in unmatched_symptoms:
                matches = process.extract(symptom, symptoms_columns, limit=3)
                partial_matches[symptom] = matches
        
        warning_message = f"Warning: Some symptoms did not match exactly. Here are some partial matches:\n" if partial_matches else "All symptoms match exactly."
        for symptom, matches in partial_matches.items():
            match_details = [f"{match} ({score}%)" for match, score in matches]
            warning_message += f" - {symptom.capitalize()}: {', '.join(match_details)}\n"
        
        second_prediction = sec_predict(symptoms_exp)
        
        X_train, X_test, y_train, y_test = train_test_split(X, df['prognosis'], test_size=0.3, random_state=20, stratify=df['prognosis'])
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
        symptoms_vector = np.isin(X.columns.str.lower(), symptoms_exp).astype(int)
        
        predicted_index = clf.predict([symptoms_vector])[0]
        
        try:
            predicted_disease = le.inverse_transform([predicted_index])[0]
        except ValueError:
            predicted_disease = "Unknown disease"
        
        # Fetch description, precautions, and severity
        total_severity, average_severity, description, precautions, severity_message = calc_condition(symptoms_exp, int(num_days), predicted_disease)
        
        # Format the response
        response = f"""
        Hello {name},
        Based on your symptoms, you may have: {predicted_disease}

        Description: {description}

        Total Severity Score: {total_severity}
        Average Severity over {num_days} days: {average_severity}

        Secondary Prediction (based on fuzzy matching): {second_prediction}

        Precautions: {"; ".join(precautions)}

        Severity Advice: {severity_message}
        """
        
        response += warning_message
        
        return response
    except Exception as e:
        return f"Error: Unable to predict disease. Please check the symptoms. ({str(e)})"

# Gradio Interface
interface = gr.Interface(
    fn=healthcare_chatbot,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Textbox(label="List of Symptoms (separate by comma)"),
        gr.Number(label="Duration of Symptoms in Days")
    ],
    outputs="text",
    title="Healthcare Symptom-Based Chatbot",
    description="Provide your name, symptoms (comma-separated), and duration of symptoms for a preliminary health assessment."
)
interface.launch(share=True)
interface.launch()


C:\Users\harshulava\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Data loaded successfully.
* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://cc087f567e9dd71873.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.


In [4]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr

# Initialize dictionaries to store severity, descriptions, and precautions
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to load severity data from CSV
def getSeverityDict():
    global severityDictionary
    with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            if len(row) < 2:
                continue
            try:
                severityDictionary[row[0]] = int(row[1])
            except ValueError:
                pass

# Function to load symptom descriptions from CSV
def getDescription():
    global description_list
    with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            description_list[row[0]] = row[1]

# Function to load precaution data from CSV
def getprecautionDict():
    global precautionDictionary
    with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            precautionDictionary[row[0]] = row[1:]

# Load data into dictionaries
getSeverityDict()
getDescription()
getprecautionDict()

# Prediction and symptom processing functions
def predict_disease(symptoms):
    df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    for symptom in symptoms:
        if symptom in symptoms_dict:
            input_vector[symptoms_dict[symptom]] = 1
    
    prediction = clf.predict([input_vector])[0]
    description = description_list.get(prediction, "No description available.")
    precautions = precautionDictionary.get(prediction, ["No precautions available."])
    
    return prediction, description, precautions

# Gradio function to interface with the chatbot
def chatbot_interface(name, symptoms):
    if not name:
        return "Please enter your name.", "", ""
    
    prediction, description, precautions = predict_disease(symptoms)
    precautions_text = "\n".join(f"{i + 1}. {precaution}" for i, precaution in enumerate(precautions))
    
    return f"Hello {name}, you may have {prediction}.", description, precautions_text

# Gradio interface layout
interface = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.CheckboxGroup(choices=list(severityDictionary.keys()), label="Select Your Symptoms")
    ],
    outputs=[
        gr.Textbox(label="Diagnosis"),
        gr.Textbox(label="Description"),
        gr.Textbox(label="Precautions")
    ],
    title="Healthcare ChatBot",
    description="A healthcare chatbot to help predict potential diseases based on symptoms."
)

# Launch the Gradio app
interface.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [4]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr

# Initialize dictionaries to store severity, descriptions, and precautions
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to load severity data from CSV
def getSeverityDict():
    global severityDictionary
    with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            if len(row) < 2:
                continue
            try:
                severityDictionary[row[0]] = int(row[1])
            except ValueError:
                pass

# Function to load symptom descriptions from CSV
def getDescription():
    global description_list
    with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            description_list[row[0]] = row[1]

# Function to load precaution data from CSV
def getprecautionDict():
    global precautionDictionary
    with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            precautionDictionary[row[0]] = row[1:]

# Load data into dictionaries
getSeverityDict()
getDescription()
getprecautionDict()

# Prediction and symptom processing functions
def predict_disease(symptoms):
    df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    for symptom in symptoms:
        if symptom in symptoms_dict:
            input_vector[symptoms_dict[symptom]] = 1
    
    prediction = clf.predict([input_vector])[0]
    description = description_list.get(prediction, "No description available.")
    precautions = precautionDictionary.get(prediction, ["No precautions available."])
    
    return prediction, description, precautions

# New function to handle login (hardcoded for simplicity)
def login(username, password):
    # In a real scenario, you would check the credentials against a database
    if username == "user" and password == "password":
        return True
    else:
        return False

# Function to handle chatbot interface after login
def chatbot_interface(username, password, symptoms, medical_history):
    if not login(username, password):
        return "Invalid login credentials.", "", ""
    
    if not symptoms:
        return "Please select your symptoms.", "", ""
    
    prediction, description, precautions = predict_disease(symptoms)
    precautions_text = "\n".join(f"{i + 1}. {precaution}" for i, precaution in enumerate(precautions))
    
    return f"Hello {username}, you may have {prediction}.", description, precautions_text

# Gradio function to ask for user inputs (login and medical history)
def gradio_interface(username, password, symptoms, medical_history):
    if not login(username, password):
        return "Invalid login credentials.", "", "", ""
    
    prediction, description, precautions = predict_disease(symptoms)
    precautions_text = "\n".join(f"{i + 1}. {precaution}" for i, precaution in enumerate(precautions))
    
    return f"Hello {username}, you may have {prediction}.", description, precautions_text, medical_history

# Gradio interface layout
interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Username", placeholder="Enter your username"),
        gr.Textbox(label="Password", type="password", placeholder="Enter your password"),
        gr.CheckboxGroup(choices=list(severityDictionary.keys()), label="Select Your Symptoms"),
        gr.Textbox(label="Medical History", placeholder="Enter your medical history (optional)")
    ],
    outputs=[
        gr.Textbox(label="Diagnosis"),
        gr.Textbox(label="Description"),
        gr.Textbox(label="Precautions"),
        gr.Textbox(label="Medical History")
    ],
    title="Healthcare ChatBot",
    description="A healthcare chatbot to help predict potential diseases based on symptoms, with login and medical history features."
)

# Launch the Gradio app
interface.launch()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [14]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr

# Initialize dictionaries to store severity, descriptions, and precautions
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to load severity data from CSV
def getSeverityDict():
    global severityDictionary
    with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            if len(row) < 2:
                continue
            try:
                severityDictionary[row[0]] = int(row[1])
            except ValueError:
                pass

# Function to load symptom descriptions from CSV
def getDescription():
    global description_list
    with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            description_list[row[0]] = row[1]

# Function to load precaution data from CSV
def getprecautionDict():
    global precautionDictionary
    with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            precautionDictionary[row[0]] = row[1:]

# Load data into dictionaries
getSeverityDict()
getDescription()
getprecautionDict()

# Prediction and symptom processing functions
def predict_disease(symptoms):
    df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    for symptom in symptoms:
        if symptom in symptoms_dict:
            input_vector[symptoms_dict[symptom]] = 1
    
    prediction = clf.predict([input_vector])[0]
    description = description_list.get(prediction, "No description available.")
    precautions = precautionDictionary.get(prediction, ["No precautions available."])
    
    return prediction, description, precautions

# Gradio function to handle symptom selection and medical history
def gradio_interface(symptoms, medical_history):
    if not symptoms:
        return "Please select your symptoms.", "", "", ""
    
    prediction, description, precautions = predict_disease(symptoms)
    precautions_text = "\n".join(f"{i + 1}. {precaution}" for i, precaution in enumerate(precautions))
    
    return f"You may have {prediction}.", description, precautions_text, medical_history

# Gradio interface layout
interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.CheckboxGroup(choices=list(severityDictionary.keys()), label="Select Your Symptoms"),
        gr.Textbox(label="Medical History", placeholder="Enter your medical history (optional)")
    ],
    outputs=[
        gr.Textbox(label="Diagnosis"),
        gr.Textbox(label="Description"),
        gr.Textbox(label="Precautions"),
        gr.Textbox(label="Medical History")
    ],
    title="Healthcare ChatBot",
    description="A healthcare chatbot to help predict potential diseases based on symptoms, with medical history features."
)

# Launch the Gradio app
interface.launch()


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [18]:
import csv
import pandas as pd
import numpy as np
import pyttsx3
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr

# Initialize dictionaries to store severity, descriptions, and precautions
le = LabelEncoder()
severityDictionary = {}
description_list = {}
precautionDictionary = {}

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to load severity data from CSV
def getSeverityDict():
    global severityDictionary
    with open(r"C:\Users\harshulava\Downloads\Symptom_severity.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            if len(row) < 2:
                continue
            try:
                severityDictionary[row[0]] = int(row[1])
            except ValueError:
                pass

# Function to load symptom descriptions from CSV
def getDescription():
    global description_list
    with open(r"C:\Users\harshulava\Downloads\symptom_Description.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            description_list[row[0]] = row[1]

# Function to load precaution data from CSV
def getprecautionDict():
    global precautionDictionary
    with open(r"C:\Users\harshulava\Downloads\symptom_precaution.csv") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            precautionDictionary[row[0]] = row[1:]

# Load data into dictionaries
getSeverityDict()
getDescription()
getprecautionDict()

# Prediction and symptom processing functions
def predict_disease(symptoms):
    df = pd.read_csv(r"C:\Users\harshulava\Downloads\Training.csv")
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    
    symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}
    input_vector = np.zeros(len(symptoms_dict))
    
    for symptom in symptoms:
        if symptom in symptoms_dict:
            input_vector[symptoms_dict[symptom]] = 1
    
    prediction = clf.predict([input_vector])[0]
    description = description_list.get(prediction, "No description available.")
    precautions = precautionDictionary.get(prediction, ["No precautions available."])
    
    return prediction, description, precautions

# Gradio function to handle symptom selection and medical history
def gradio_interface(name, days, symptoms, medical_history):
    if not symptoms:
        return "Please select your symptoms.", "", "", "", ""
    
    prediction, description, precautions = predict_disease(symptoms)
    precautions_text = "\n".join(f"{i + 1}. {precaution}" for i, precaution in enumerate(precautions))
    
    return f"Hello {name}, you may have {prediction}.", description, precautions_text, medical_history, f"Symptoms duration: {days} days"

# Gradio interface layout
interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Your Name", placeholder="Enter your name"),
        gr.Number(label="Days with Symptoms", precision=0, value=0),  # Use 'value' instead of 'placeholder' for the number input
        gr.CheckboxGroup(choices=list(severityDictionary.keys()), label="Select Your Symptoms"),
        gr.Textbox(label="Medical History", placeholder="Enter your medical history (optional)")
    ],
    outputs=[
        gr.Textbox(label="Diagnosis"),
        gr.Textbox(label="Description"),
        gr.Textbox(label="Precautions"),
        gr.Textbox(label="Medical History"),
        gr.Textbox(label="Symptoms Duration")
    ],
    title="Healthcare ChatBot",
    description="A healthcare chatbot to help predict potential diseases based on symptoms, with medical history features."
)

# Launch the Gradio app
interface.launch()


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
